In [2]:
# 参考https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/5-09-RNN3/
from sklearn.datasets import load_boston
from sklearn import preprocessing
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [3]:
%matplotlib qt
tf.reset_default_graph()
# 波士顿房价数据
boston = load_boston()
x = boston.data
y = boston.target

print('波士顿数据X:', x.shape)  # (506, 13)
# print(x[::100])
print('波士顿房价Y:', y.shape)
# print(y[::100])
# 数据标准化
ss_x = preprocessing.StandardScaler()
train_x = ss_x.fit_transform(x)
# print(train_x)
ss_y = preprocessing.StandardScaler()
train_y = ss_y.fit_transform(y.reshape(-1, 1))

BATCH_START = 0  # 建立 batch data 时候的 index
TIME_STEPS = 10  # backpropagation through time 的 time_steps
BATCH_SIZE = 30
INPUT_SIZE = 13  
OUTPUT_SIZE = 1  
CELL_SIZE = 10  # RNN 的 hidden unit size
LR = 0.006  # learning rate
# 保存session状态的位置
save_dir = './bst_save/ckpt/bst'
# logs_path = './bst_save/log/'

def get_batch_boston():
    global train_x, train_y, BATCH_START, TIME_STEPS
    # 每次都是拿300行，13列的数据，但是起始点根据 BATCH_START
    x_part1 = train_x[BATCH_START:BATCH_START + TIME_STEPS * BATCH_SIZE]
    y_part1 = train_y[BATCH_START:BATCH_START + TIME_STEPS * BATCH_SIZE]
    # 时间段= 0 300
    # x_part1= (300, 13)
    # 时间段= 10 310
    # x_part1= (300, 13)
#     print('时间段=', BATCH_START, BATCH_START + TIME_STEPS * BATCH_SIZE)
#     print('x_part1=', x_part1.shape)
#     print(BATCH_SIZE, TIME_STEPS, INPUT_SIZE)
    # 转成 30 10 13的形状
    seq = x_part1.reshape((BATCH_SIZE, TIME_STEPS, INPUT_SIZE))
    res = y_part1.reshape((BATCH_SIZE, TIME_STEPS, 1))

    BATCH_START += TIME_STEPS

    # returned seq, res and xs: shape (batch, step, input)
    #np.newaxis 用来增加一个维度 变为三个维度，第三个维度将用来存上一批样本的状态
    return [seq, res]


# def get_batch():
#     global BATCH_START, TIME_STEPS
#     # xs shape (50batch, 20steps)
#     xs = np.arange(BATCH_START, BATCH_START + TIME_STEPS * BATCH_SIZE).reshape(
#         (BATCH_SIZE, TIME_STEPS)) / (10 * np.pi)
#     print('xs.shape=', xs.shape)
#     seq = np.sin(xs)
#     res = np.cos(xs)
#     BATCH_START += TIME_STEPS
#     # import matplotlib.pyplot as plt
#     # plt.plot(xs[0, :], res[0, :], 'r', xs[0, :], seq[0, :], 'b--')
#     # plt.show()
#     print('增加维度前:', seq.shape)
#     print(seq[:2])
#     print('增加维度后:', seq[:, :, np.newaxis].shape)
#     print(seq[:2])
#     # returned seq, res and xs: shape (batch, step, input)
#     #np.newaxis 用来增加一个维度 变为三个维度，第三个维度将用来存上一批样本的状态
#     return [seq[:, :, np.newaxis], res[:, :, np.newaxis], xs]


class LSTMRNN(object):
    def __init__(self, n_steps, input_size, output_size, cell_size,
                 batch_size):
        '''
        :param n_steps: 每批数据总包含多少时间刻度
        :param input_size: 输入数据的维度
        :param output_size: 输出数据的维度 如果是类似价格曲线的话，应该为1
        :param cell_size: cell的大小
        :param batch_size: 每批次训练数据的数量
        '''
        self.n_steps = n_steps
        self.input_size = input_size
        self.output_size = output_size
        self.cell_size = cell_size
        self.batch_size = batch_size
        with tf.name_scope('inputs'):
            self.xs = tf.placeholder(
                tf.float32, [None, n_steps, input_size], name='xs')  #xs 有三个维度
            self.ys = tf.placeholder(
                tf.float32, [None, n_steps, output_size], name='ys')  #ys 有三个维度
        with tf.variable_scope('in_hidden'):
            self.add_input_layer()
        with tf.variable_scope('LSTM_cell'):
            self.add_cell()
        with tf.variable_scope('out_hidden'):
            self.add_output_layer()
        with tf.name_scope('cost'):
            self.compute_cost()
        with tf.name_scope('train'):
            self.train_op = tf.train.AdamOptimizer(LR).minimize(self.cost)

    #增加一个输入层
    def add_input_layer(self, ):
        # l_in_x:(batch*n_step, in_size),相当于把这个批次的样本串到一个长度1000的时间线上，每批次50个样本，每个样本20个时刻
        l_in_x = tf.reshape(
            self.xs, [-1, self.input_size], name='2_2D')  #-1 表示任意行数
        # Ws (in_size, cell_size)
        Ws_in = self._weight_variable([self.input_size, self.cell_size])
        # bs (cell_size, )
        bs_in = self._bias_variable([
            self.cell_size,
        ])
        # l_in_y = (batch * n_steps, cell_size)
        with tf.name_scope('Wx_plus_b'):
            l_in_y = tf.matmul(l_in_x, Ws_in) + bs_in
        # reshape l_in_y ==> (batch, n_steps, cell_size)
        self.l_in_y = tf.reshape(
            l_in_y, [-1, self.n_steps, self.cell_size], name='2_3D')

    #多时刻的状态叠加层 CELL
    def add_cell(self):
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(
            self.cell_size, forget_bias=1.0, state_is_tuple=True)
        with tf.name_scope('initial_state'):
            self.cell_init_state = lstm_cell.zero_state(
                self.batch_size, dtype=tf.float32)
        #time_major=False 表示时间主线不是第一列batch
        self.cell_outputs, self.cell_final_state = tf.nn.dynamic_rnn(
            lstm_cell,
            self.l_in_y,
            initial_state=self.cell_init_state,
            time_major=False)

    # 增加一个输出层
    def add_output_layer(self):
        # shape = (batch * steps, cell_size)
        l_out_x = tf.reshape(
            self.cell_outputs, [-1, self.cell_size], name='2_2D')
        Ws_out = self._weight_variable([self.cell_size, self.output_size])
        bs_out = self._bias_variable([
            self.output_size,
        ])
        # shape = (batch * steps, output_size)
        with tf.name_scope('Wx_plus_b'):
            self.pred = tf.matmul(l_out_x, Ws_out) + bs_out  #预测结果
    
    # 计算损失
    def compute_cost(self):
        losses = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
            [tf.reshape(self.pred, [-1], name='reshape_pred')],
            [tf.reshape(self.ys, [-1], name='reshape_target')],
            [tf.ones([self.batch_size * self.n_steps], dtype=tf.float32)],
            average_across_timesteps=True,
            softmax_loss_function=self.ms_error,
            name='losses')
        with tf.name_scope('average_cost'):
            self.cost = tf.div(
                tf.reduce_sum(losses, name='losses_sum'),
                self.batch_size,
                name='average_cost')
            tf.summary.scalar('cost', self.cost)

    def ms_error(self, labels, logits):
        return tf.square(tf.subtract(labels, logits))

    def _weight_variable(self, shape, name='weights'):
        initializer = tf.random_normal_initializer(
            mean=0.,
            stddev=1.,
        )
        return tf.get_variable(shape=shape, initializer=initializer, name=name)

    def _bias_variable(self, shape, name='biases'):
        initializer = tf.constant_initializer(0.1)
        return tf.get_variable(name=name, shape=shape, initializer=initializer)



波士顿数据X: (506, 13)
波士顿房价Y: (506,)


In [4]:
# 训练
tf.reset_default_graph()
if __name__ == '__main__':
    # 设置画布初始属性和内容

    fig = plt.figure(figsize=(20, 3))  # dpi参数指定绘图对象的分辨率，即每英寸多少个像素，缺省值为80
    axes = fig.add_subplot(1, 1, 1)
    train_y1 = train_y[190:490]
    # 限制100个显示值
#     line3, = axes.plot(range(100), train_y1.flatten()[-100:], 'r', label='实际')
    line3, = axes.plot(train_y1.flatten(), 'r', label='实际')
    plt.ion()
    plt.show()
    
    seq, res = get_batch_boston()
    model = LSTMRNN(TIME_STEPS, INPUT_SIZE, OUTPUT_SIZE, CELL_SIZE, BATCH_SIZE)
    sess = tf.Session()
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("logs", sess.graph)
    # tf.initialize_all_variables() no long valid from
    # 2017-03-02 if using tensorflow >= 0.12
    sess.run(tf.global_variables_initializer())
    # relocate to the local dir and run this line to view it on Chrome (http://0.0.0.0:6006/):
    # $ tensorboard --logdir='logs'
    for j in range(2):  #训练200次
        pred_res = None
        for i in range(20):  #把整个数据分为20个时间段
            seq, res = get_batch_boston()
#             print('seq',seq.shape)
            if i == 0:
                feed_dict = {
                    model.xs: seq,
                    model.ys: res,
                    # create initial state
                }
            else:
                feed_dict = {
                    model.xs: seq,
                    model.ys: res,
                    model.cell_init_state:
                    state  # use last state as the initial state for this run
                }
#             print('seq',seq.shape)
            _, cost, state, pred = sess.run([
                model.train_op, model.cost, model.cell_final_state, model.pred
            ],feed_dict=feed_dict)
            pred_res = pred

            result = sess.run(merged, feed_dict)
            writer.add_summary(result, i)
            
        print('{0} cost: '.format(j), round(cost, 4))
        BATCH_START = 0  #从头再来一遍

        # 画图
#         print("结果:", pred_res.shape)
        #与最后一次训练所用的数据保持一致
       
#         print('实际', train_y1.flatten().shape)

    #     r_size = BATCH_SIZE * TIME_STEPS
        ###画图###########################################################################
        
        #为了方便看，只显示了后100行数据
        try:
            axes.lines.remove(line1)
        except Exception:
            pass
#         handles, labels = plt.gca().get_legend_handles_labels()
        # 返回是, 是一个Line2D的对象，无逗号是一个包含了Line2D的数组
#         line1, = axes.plot(
#             range(100), pred.flatten()[-100:], 'b--', label='rnn计算结果 '+str(j))
        line1, = axes.plot(pred.flatten(), 'b--', label='rnn计算结果 '+str(j))
    #     line2,=axes.plot(range(len(gbr_pridict)), gbr_pridict, 'r--',label='优选参数')
#         plt.draw()
        axes.grid()
        fig.tight_layout()
        #plt.legend(handles=[line1, line2,line3])
#         plt.legend(handles=handles)
        plt.legend(handles=[line1,line3])
        plt.title('递归神经网络')
    #     plt.show()
        plt.pause(0.5)
    plt.ioff()
    plt.show()
    
    
    # 保存模型
    saver = tf.train.Saver()
    saver.save(sess, save_dir,)
    print('Model Trained and Saved')

0 cost:  4.9066
1 cost:  2.7821
Model Trained and Saved


In [9]:
%matplotlib qt
gen_length = 500
prime_word = '1'
loaded_graph = tf.Graph()
tf.reset_default_graph()
seq, res = get_batch_boston()
model = LSTMRNN(TIME_STEPS, INPUT_SIZE, OUTPUT_SIZE, CELL_SIZE, BATCH_SIZE)
saver = tf.train.Saver()
fig = plt.figure(figsize=(20, 3))  # dpi参数指定绘图对象的分辨率，即每英寸多少个像素，缺省值为80
axes = fig.add_subplot(1, 1, 1)
train_y1 = train_y[190:490]
train_y2 = train_y[0:300]
train_y3 = train_y[0:506]
line5, = axes.plot(train_y3.flatten(), 'y', label='实际3')
# line3, = axes.plot(train_y1.flatten(), 'r', label='实际1')
# line4, = axes.plot(train_y2.flatten(), 'g', label='实际2')
plt.ion()
plt.show()
print('seq', seq.shape)
with tf.Session() as sess:
    # 加载保存过的session
    #     loader = tf.train.import_meta_graph(save_dir + '.meta')
    saver.restore(sess, './bst_save/ckpt/bst')
    #取训练集的最后一行，作为开始
    prev_seq = seq
    #     print(type(seq))
    #     print(seq.shape)
    #     print(prev_seq.shape)
    #     print(seq)
    #     print(prev_seq)
    #     Dimensions of inputs should match: shape[0] = [1,10] vs. shape[1] = [30,10]
    #     predict=[]
    #     得到之后100个预测结果
    for i in range(20):
        if i == 1:
            print('if')
            feed_dict = {
                # create initial state
                model.xs: seq  
            }
        else:
            print('else')
            feed_dict = {
                model.xs: seq,
                # use last state as the initial state for this run
                model.cell_init_state: state 
            }
        state, pred = sess.run([model.cell_final_state, model.pred],
                               feed_dict=feed_dict)
#         print(type(next_seq))
#         print('next_seq', next_seq.shape)
        #         print('next_seq',state.shape)
#         next_seq_re = tf.reshape(next_seq, [30, 10])
        #         predict.append(next_seq[-1])
        #         print(prev_seq.shape)
        #         print(next_seq.shape)
        #         print(next_seq)
        #         print(next_seq[-1])
        #每次得到最后一个时间步的预测结果，与之前的数据加在一起，形成新的测试样本
        # 需要把新的预测值加入，然后重新预测，如此循环
        #         prev_seq=np.vstack((prev_seq[1:],next_seq[-1]))

        try:
            axes.lines.remove(line1)
        except Exception:
            pass
        line1, = axes.plot(next_seq.flatten(), 'b--', label='rnn计算结果' + str(i))
        axes.grid()
        fig.tight_layout()
        #plt.legend(handles=[line1, line2,line3])
        #         plt.legend(handles=handles)
        plt.legend(handles=[line1, line3])
        plt.title('递归神经网络')
        #     plt.show()
        plt.pause(0.5)

    #     plt.plot(list(range(len(normalize_data), len(normalize_data) + len(predict))), predict, color='r')
    BATCH_START = 0
    plt.ioff()
    plt.show()

时间段= 10 310
x_part1= (300, 1)
30 10 1
seq (30, 10, 1)
INFO:tensorflow:Restoring parameters from ./bst_save/ckpt/bst
else


NameError: name 'next_seq' is not defined

# 

In [30]:
# %matplotlib qt
gen_length = 500
prime_word = '1'
loaded_graph = tf.Graph()
tf.reset_default_graph()
seq, res = get_batch_boston()
model = LSTMRNN(TIME_STEPS, INPUT_SIZE, OUTPUT_SIZE, CELL_SIZE, BATCH_SIZE)
saver = tf.train.Saver()
fig = plt.figure(figsize=(20, 3))  # dpi参数指定绘图对象的分辨率，即每英寸多少个像素，缺省值为80
axes = fig.add_subplot(1, 1, 1)
train_y1 = train_y[190:490]
train_y2 = train_y[0:300]
train_y3 = train_y
# train_y1 = train_y[190:300]
# train_y2 = train_y[190:300]

print(type(train_y1))
print(train_y1.shape)
# train_y1.flatten()[-100:]
# line3, = axes.plot(train_y1.flatten(), 'r', label='实际1>190')

line5, = axes.plot(train_y3.flatten(), 'y', label='实际3')
line4, = axes.plot(train_y2.flatten(), 'g', label='实际2>0')
plt.legend(handles=[line4,line3])
plt.ion()
plt.show()

时间段= 170 470
x_part1= (300, 13)
<class 'numpy.ndarray'>
(300, 1)


In [133]:
a=np.arange(4).reshape(2,2)
print('a',a)

print('a1',a[1])

print('a',a.shape)
print('a1',a[1].shape)

a [[0 1]
 [2 3]]
a1 [2 3]
a (2, 2)
a1 (2,)


In [131]:
a=np.arange(6).reshape(3,2)
print(a)
print(a.flatten())
print(a.flatten()[-2:])

[[0 1]
 [2 3]
 [4 5]]
[0 1 2 3 4 5]
[4 5]


In [32]:
plt.figure()

plt.subplot(2,1,1)      # 分成两行一列，起始点为1
plt.plot([0,1],[0,1])   # 设置xy轴范围

plt.subplot(2,3,4)      # 分成两行三列，起始点位4
plt.plot([0,1],[0,2])

plt.subplot(2,3,5)
plt.plot([0,1],[0,3])

plt.subplot(2,3,6)
plt.plot([0,1],[0,4])

plt.show()

In [10]:

a=np.arange(56).reshape(8,7)
#X取前5行的前三列（从1计数）
x=a[0:0+5,:3]
#Y取前5行的第4列（从0计数），并行转列
y=a[0:0+5,0,np.newaxis]
z=a[:,0]
print(a)
print(z)
zz =  (a[:,0] * 1000 / 60) - (a[:,4] + a[:,5] + a[:,6])
print(zz)
print('a',a[1:])
# print(x.shape)
# print(x)
# print(y.shape)
# print(y)

[[ 0  1  2  3  4  5  6]
 [ 7  8  9 10 11 12 13]
 [14 15 16 17 18 19 20]
 [21 22 23 24 25 26 27]
 [28 29 30 31 32 33 34]
 [35 36 37 38 39 40 41]
 [42 43 44 45 46 47 48]
 [49 50 51 52 53 54 55]]
[ 0  7 14 21 28 35 42 49]
[-15.          80.66666667 176.33333333 272.         367.66666667
 463.33333333 559.         654.66666667]
a [[ 7  8  9 10 11 12 13]
 [14 15 16 17 18 19 20]
 [21 22 23 24 25 26 27]
 [28 29 30 31 32 33 34]
 [35 36 37 38 39 40 41]
 [42 43 44 45 46 47 48]
 [49 50 51 52 53 54 55]]


In [40]:
train_x,train_y=[],[]
for i in range(len(y)-TIME_STEPS-1):
    x1=y[i:i+TIME_STEPS]
    y1=y[i+1:i+TIME_STEPS+1]
    train_x.append(x1.tolist())
    train_y.append(y1.tolist()) 

In [50]:
print(len(train_x[-1]))
print(len(train_x[0:10]))
print(train_x[0:10])

10
10
[[24.0, 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9], [21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15.0], [34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15.0, 18.9], [33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15.0, 18.9, 21.7], [36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15.0, 18.9, 21.7, 20.4], [28.7, 22.9, 27.1, 16.5, 18.9, 15.0, 18.9, 21.7, 20.4, 18.2], [22.9, 27.1, 16.5, 18.9, 15.0, 18.9, 21.7, 20.4, 18.2, 19.9], [27.1, 16.5, 18.9, 15.0, 18.9, 21.7, 20.4, 18.2, 19.9, 23.1], [16.5, 18.9, 15.0, 18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5], [18.9, 15.0, 18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2]]
